In [1]:
import sys
sys.path.append('../utils/')
sys.path.append('../paviaUTools/')

import matplotlib.pyplot as plt
from datasetLoader import datasetLoader
import os
import numpy as np
from whole_pipeline import whole_pipeline_all, whole_pipeline_divided, whole_pipeline_divided_parallel
import torch
from plots import *
from weights_anal import *

import gc
torch.cuda.empty_cache()
gc.collect()

%matplotlib inline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
parent_dir = os.path.join(os.getcwd(),"..")
csv_path = os.path.join(parent_dir, 'datasets', 'paviaU.csv')
gt_path = os.path.join(parent_dir, 'datasets', 'paviaU_gt.csv')
# csv_path = os.path.join(parent_dir, 'datasets', 'pavia.csv')
# gt_path = os.path.join(parent_dir, 'datasets', 'pavia_gt.csv')

dsl = datasetLoader(csv_path, gt_path)

df = dsl.read_dataset(gt=False)
X = np.array(df)
X = X.reshape((610,340, 103))
# X = X.reshape((1096, 715, 102))

df = dsl.read_dataset(gt=True)
y = np.array(df)

factor = 29
rows_factor = factor
cols_factor = factor

X = torch.from_numpy(X)
y = torch.from_numpy(y)

X = X.to(device)
y = y.to(device)

In [3]:
reps = 10

In [4]:
import random
import numpy as np

# 32-bit integer
int32_max = np.iinfo(np.int32).max
int32_min = np.iinfo(np.int32).min

random_seeds = [random.randint(int32_min, int32_max) for _ in range (reps)]

random_seeds

[609871260,
 -1251577339,
 1951047638,
 1369137073,
 625047430,
 -952091326,
 1801071992,
 1971026286,
 488610184,
 -1133543652]

In [5]:
import gc
from MetaLearner import HDDOnBands

#Naive component

avg_acc_train = 0.0
avg_acc_test = 0.0

for i in range(reps):
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_all(X,y, rows_factor, cols_factor, is_normalize_each_band=True, method_label_patch='most_common', random_seed=random_seeds[i])
    avg_acc_train += train_acc/reps
    avg_acc_test += test_acc/reps

    print("iteration ", i, " stats: ")
    print("train_acc: ", train_acc)
    print("test_acc: ", test_acc)

print("avg_acc_train: ", avg_acc_train)
print("avg_acc_test: ", avg_acc_test)



XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  0.5566866397857666
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  35292
total classified:  7483
Train Accuracy:  0.8894933696021762
Test Accuracy:  0.5611385807831084
iteration  0  stats: 
train_acc:  0.8894933696021762
test_acc:  0.5611385807831084
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  0.46994590759277344
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  36726
total classified:  6049
Train Accuracy:  0.8930457986167838
Test Accuracy:  0.4704909902463217
iteration  1  stats: 
train_acc:  0.8930457986167838
test_acc:  0.4704909902463217
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  0.5003552436828613
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  35003
total classified:  7772
Train Accuracy:  0.8899237208239293
Test Accuracy:  0.5292074112197632
iteration  2  stats: 
train_acc:  0.8899237208239293
test_acc:  0.5292074112197632
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  0.5882153511047363
XXXXXXX IN CLA

In [6]:
#Partition componenets
clusters_amount = 2

In [7]:
import gc
from MetaLearner import HDDOnBands

#Random Partition component

avg_acc_train = 0.0
avg_acc_test = 0.0

for i in range(reps):
    weights, dist_batches = HDDOnBands.createUniformWeightedBatches(X, clusters_amount=clusters_amount, random_seed=random_seeds[i])
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_divided_parallel(X,y, rows_factor, cols_factor, is_normalize_each_band=True, method_label_patch='most_common', weights=weights, distance_batches= dist_batches, random_seed=random_seeds[i])
    avg_acc_train += train_acc/reps
    avg_acc_test += test_acc/reps

    print("iteration ", i, " stats: ")
    print("train_acc: ", train_acc)
    print("test_acc: ", test_acc)

print("avg_acc_train: ", avg_acc_train)
print("avg_acc_test: ", avg_acc_test)


TOTAL TIME FOR METHOD:  5.705860614776611
Train Accuracy:  0.8894933696021762
Test Accuracy:  0.5104904450086863
iteration  0  stats: 
train_acc:  0.8894933696021762
test_acc:  0.5104904450086863
TOTAL TIME FOR METHOD:  5.266321182250977
Train Accuracy:  0.8930457986167838
Test Accuracy:  0.4600760456273764
iteration  1  stats: 
train_acc:  0.8930457986167838
test_acc:  0.4600760456273764
TOTAL TIME FOR METHOD:  5.501449346542358
Train Accuracy:  0.8899237208239293
Test Accuracy:  0.48700463201235206
iteration  2  stats: 
train_acc:  0.8899237208239293
test_acc:  0.48700463201235206
TOTAL TIME FOR METHOD:  7.020761489868164
Train Accuracy:  0.8777624741751706
Test Accuracy:  0.6532459137501154
iteration  3  stats: 
train_acc:  0.8777624741751706
test_acc:  0.6532459137501154
TOTAL TIME FOR METHOD:  6.030987501144409
Train Accuracy:  0.8816484478716493
Test Accuracy:  0.5345903418339664
iteration  4  stats: 
train_acc:  0.8816484478716493
test_acc:  0.5345903418339664
TOTAL TIME FOR MET

In [8]:
import gc
from MetaLearner import HDDOnBands

#Similarity based Partition component

avg_acc_train = 0.0
avg_acc_test = 0.0

for i in range(reps):
    weights, dist_batches = HDDOnBands.classicUnsurpervisedClustering(X, clusters_amount=clusters_amount)
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_divided_parallel(X,y, rows_factor, cols_factor, is_normalize_each_band=True, method_label_patch='most_common', weights=weights, distance_batches= dist_batches, random_seed=random_seeds[i])
    avg_acc_train += train_acc/reps
    avg_acc_test += test_acc/reps

    print("iteration ", i, " stats: ")
    print("train_acc: ", train_acc)
    print("test_acc: ", test_acc)

print("avg_acc_train: ", avg_acc_train)
print("avg_acc_test: ", avg_acc_test)


AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

: 

In [ ]:
import gc
from MetaLearner import HDDOnBands

#Regrouping Partition component

avg_acc_train = 0.0
avg_acc_test = 0.0

for i in range(reps):
    weights, dist_batches = HDDOnBands.regroupingUnsurpervisedClusters(X, clusters_amount=clusters_amount)
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_divided_parallel(X,y, rows_factor, cols_factor, is_normalize_each_band=True, method_label_patch='most_common', weights=weights, distance_batches= dist_batches, random_seed=random_seeds[i])
    avg_acc_train += train_acc/reps
    avg_acc_test += test_acc/reps

    print("iteration ", i, " stats: ")
    print("train_acc: ", train_acc)
    print("test_acc: ", test_acc)

print("avg_acc_train: ", avg_acc_train)
print("avg_acc_test: ", avg_acc_test)
